In [5]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

# import time
# # tmp sleep
# time.sleep(3600)
# print('sleep done')

# load feats
train_x,test_x = [],[]
for feat in glob.glob('../features/*.pkl'):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
file path ../features/lstm_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/simple_rnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat1.pkl
(95851, 30) (226998, 30)
file path ../features/lstm_muse_adj_bn_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_fea

In [6]:
print(train_y[:10])
print(len(train_y))
print(sum(train_y[:,0]))

[[1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
95851
9237


In [7]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        
        # train for each class
        for i in range(6):
            if model_name == 'lr':
                model = LogisticRegression()
            if model_name == 'xgb':
                model = XGBClassifier()
            model.fit(curr_x,curr_y[:,i])
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict_proba(curr_x))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict_proba(hold_out_x))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')
            
                
            

done


In [8]:
xgb_res = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_all_ens_csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0
0.0802385102885 0.0853102923404
1
0.0166474257116 0.0221620576982
2
0.0412069284547 0.0449458283017
3
0.00393746099722 0.00890110121129
4
0.0528463095294 0.061982745603
5
0.0148527043753 0.0194547668902
this fold avg train 0.0349548898928 avg val 0.0404594653408
0
0.0789062043402 0.0890708561894
1
0.0169995381757 0.0216149174619
2
0.0401284231427 0.047310751776
3
0.00453242093033 0.00864228540978
4
0.0543269802579 0.0590501406144
5
0.0147141019552 0.0205942158001
this fold avg train 0.034934611467 avg val 0.0410471945419
0
0.0796609836846 0.087504035516
1
0.0165787696331 0.0222064504768
2
0.0396541438242 0.0494397942444
3
0.00448536976647 0.00761159097904
4
0.0538729219347 0.0602342281945
5
0.0150691310288 0.0198980651634
this fold avg train 0.0348868866453 avg val 0.041149027429
all train avg 0.0349254626684 all val avg 0.0408852291039
         id     toxic  severe_toxic   obscene    threat    insult  \
0   6044863  0.001961      0.000066  0.000590  0.000061  0.000598   
1   6102620

In [9]:
xgb_res = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_all_ens_csv_5fold.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0
0.081112397026 0.0841433369316
1
0.0171521802247 0.0217213841612
2
0.0420741039343 0.0430105092092
3
0.00466402482066 0.00870923143757
4
0.0543606324963 0.0602328311571
5
0.0154930191612 0.0187666620873
this fold avg train 0.0358093929439 avg val 0.039430659164
0
0.0802070699331 0.0888963194489
1
0.017316366143 0.0219485458721
2
0.0408982053084 0.0462412544145
3
0.00456525757725 0.00921678367463
4
0.0540676058163 0.0604876235101
5
0.0155973693431 0.0194545358238
this fold avg train 0.0354419790202 avg val 0.0410408437907
0
0.0807999573759 0.0861057268456
1
0.0172582460555 0.0214223448163
2
0.0407603802811 0.0478919478447
3
0.00484019399631 0.00809522395536
4
0.0545835243302 0.060995334687
5
0.0154983179446 0.0204936514011
this fold avg train 0.0356234366639 avg val 0.0408340382584
0
0.0806826650353 0.0863137823683
1
0.0169581643415 0.0231125159061
2
0.0407911453209 0.0483707486696
3
0.00449608155726 0.00930069739451
4
0.0541741761621 0.0607512587413
5
0.0154394099807 0.0200976482754
